In [1]:
import pandas as pd
import PyPDF2
import fitz
from io import StringIO

In [8]:
# This script is to read all characthers on a specific page on a PDF and Export into an Excel File

pdf_file = open('POL140298453100_smallpdf.pdf', 'rb')
pdf_reader = PyPDF2.PdfReader(pdf_file)

text = ''
for page in range(len(pdf_reader.pages)):
    text += pdf_reader.pages[0].extract_text()

df = pd.read_csv(StringIO(text), sep='\t', header=None)

df.to_excel('output.xlsx', index=False, header=False)

In [19]:
import PyPDF2

# Open the PDF file
with open('POL140298453100_smallpdf.pdf', 'rb') as file:
    # Create a PDF reader object
    pdf_reader = PyPDF2.PdfReader(file)

    # Get the first page of the PDF
    page = pdf_reader.pages[0]

    # Check if the page has any annotations
    if '/Annots' in page:
        # Loop through the annotations
        for annotation in page['/Annots']:
            # Check if the annotation is a rectangle
            if annotation['/Subtype'] == '/Square':
                print('Found an annotation rectangle!')
                break
    else:
        print('No annotations found.')


TypeError: 'IndirectObject' object is not subscriptable

In [ ]:
doc = fitz.open('POL140298453100_smallpdf.pdf')
pageone = doc[0]
words = pageone.get_text("words")

In [ ]:
def make_text(words):
    line_dict = {} 
    words.sort(key=lambda w: w[0])
    for w in words:  
        y1 = round(w[3], 1)  
        word = w[4] 
        line = line_dict.get(y1, [])  
        line.append(word)  
        line_dict[y1] = line  
    lines = list(line_dict.items())
    lines.sort()  
    return "n".join([" ".join(line[1]) for line in lines])

In [ ]:
first_annots = []
rec = pageone.first_annot.rect

#Information of words in first object is stored in mywords

mywords = [w for w in words if fitz.Rect(w[:4]) in rec]
ann = make_text(mywords)
first_annots.append(ann)

In [ ]:
print(rec)
print(first_annots)

Rect(155.52999877929688, 216.6099853515625, 286.2099914550781, 237.91998291015625)
['6862610414nTeléfono:']
